In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**1. 데이터셋 확인**

>     1.1 Null data Check
    1.2 Target label 확인
    
**2. Exploratory data analysis**

>     2.1 Pclass
    2.2 Sex
    2.3 Both Sex and Pclass
    2.4 Age
    2.5 Pclass, Sex, Age
    2.6 Embarked
    2.7 Family - SibSp(형제 자매) + Parch(부모, 자녀)
    2.8 Cabin
      

만약 데이터 사이언스, 머신러닝 또는 캐글에서 어떤 것을 해야하는지 잘 모르는 newbie라면, 타이타닉을 하시는게 가장 좋은 선택입니다.

타이타닉은 아시다시피,사상 최대 해난사고로써, 1,500여명의 희생자가 생겼습니다.
우리는 이 튜토리얼에서 타이타닉에 탑승한 사람들의 신상정보를 활용하여, 승선한 사람들의 생존여부를 예측하는 모델을 생성할 것입니다.

본 튜토리얼에서는 여러 시각화 도구 (matplotlib, seaborn, plotly), 데이터 분석도구 (pandas, numpy), 머신러닝 도구(sklear)을 사용할 것입니다.

본 튜토리얼은 캐글에 있는 여러 커널들을 참조하여 만들었습니다. 본 튜토리얼을 공부하신 뒤에, 캐글 타이타닉 컴퍼티션에 존재하는 여러 다른 커널들을 더 공부하시면 됩니다.

본 튜터리얼을 파이썬 문법에 어려움이 없으셔야 수월할 것입니다. 여기서 사용하는 라이브러리들을 써본 경험이 있으면 좋겠지만, 경험이 없다하더라도 한 줄씩 천천히 적어나가시면 충분히 하실 수 있습니다.    
      

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5)
# 이 두줄을 matplotlib 의 기본 scheme 말고 seaborn scheme을 세팅하고, 일일이 graph의 font size를 지정할 필요없이 seaborn 의  font_scale을 사용하면 편하다.

import missingno as msno

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

**앞으로 해야할 프로세스**
> 1. 데이터 셋 확인 = 데부분의 캐글 데이터는 잘 정제되어 있다. 하지만 가끔 null data가 존재하는데 이를 확인하고 수정해야한다.
2. 탐색적 데이터 분석 = 어러 feature들을 개별적으로 분석하고, feature들 간의 상관관계를 확인해야한다. 여러 시각화 툴을 사용해 insight를 얻는다.
3. feature engineering = 모델을 세우기에 앞서, 모델의 성능을 높일수 있도록 feature 들을 engineering 한다. one-hot encoding, class로 나누기, 구간으로 나누기, 텍스트 데이터 처리 등을 한다.
4. model 만들기 = sklearn 을 사용해 모델을 만든다. 파이썬에서 머신러닝을 할 때는 sklearn을 사용하면 수많은 알고리즘을 일관된 문법으로 사용할 수 있다. 물론 딥러닝을 위해 tensorflow, pytorch 등을 사용 할 수도 있다.
5. 모델 학습 및 예측 = traeinset을 가지고 모델을 학습시킨 후, testset을 가지고 prediction한다.
6. 모델 평가 = 예측 성능이 원하는 수준인지 판단한다. 풀려는 문제에 따라 모델을 평가하는 방식도 달라진다. 학습도니 모델이 어떤것을 학습하였는지 확인해본다.
    

**1. Data Set 확인**

> 파이썬에서 테이블화 된 데이터를 다루는 데 가장 최적화 되어있으며, 많이 쓰이는 라이브러리는 pandas 입니다.

> 우리는 pandas 를 사용하여 데이터 셋의 간단한 통계적 분석 부터, 복잡한 처리들을 간단한 메소드를 사용하여 해낼 수 있습니다.

> 파이썬으로 데이터 분석을 한다고 하면 반드시 능숙해져야 할 라이브러리이니, 여러 커널들을 공부하시면서 사용법에 익숙해지도록 반복 또 반복하시길 권장합니다.

> 캐글에서 데이터 셋은 보통 train, testset 으로 나뉘어 있습니다.

In [ ]:
df_train = pd.read_csv('../input/titanic/train.csv')
df_test = pd.read_csv('../input/titanic/test.csv')

In [ ]:
df_train.head()

우리가 다루는 문제에서 feature은 Pclass, Age, SibSp, Parch, Fare이며, 예측하려는  target label은 Survived이다.

In [ ]:
df_train.describe()

pandas 의 describe 함수를 통해 수학적 통계 값을 확인한다. 이때 age의 카운트가 714이고 나머지는 891인것을 보아 age에 null값이 존재함을 알 수 있다.

In [ ]:
df_test.describe()

**1.1 Null Data Check**

In [ ]:
for col in df_train.columns:
    msg = 'column: {:>11}\t Percent of nan value: {:.2f}%'.format(col,100*(df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)

In [ ]:
for col in df_test.columns:
    msg = 'column: {:>10}\t Percent of nan value: {:.2f}%'.format(col,100*(df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

In [ ]:
msno.matrix(df=df_train.iloc[:,:], figsize=(8,8),color=(0.8,0.5,0.2))

In [ ]:
msno.bar(df=df_train.iloc[:,:], figsize=(8,8),color=(0.8,0.5,0.2))

In [ ]:
msno.bar(df=df_test.iloc[:,:], figsize=(8,8),color=(0.8,0.5,0.2))

**1.2 Target Label 확인**

> target label이 어떤 distribution을 가지고 있는 지 확인해봐야 한다.

> 지금같은 binavry classification 문제의 경우에서, 1과 0의 분포가 어떠냐에 따라 모델의 평가 방법이 달라 질 수 있다.

In [ ]:
f, ax = plt.subplots(1,2,figsize=(18,8))

df_train['Survived'].value_counts().plot.pie(explode=[0,0.1], autopct='%1.1f%%', ax=ax[0],shadow=True)
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')
sns.countplot('Survived', data=df_train,ax=ax[1])
ax[1].set_title('Count plot - Survived')

plt.show()

> traget olabel 의 분포가 제법 균일 하다. 불균일한 경우, 예를 들어서 100중 1이 99 0이 1이라 해도 정확도가 99% 나오게 된다. 0을 찾는 문제라면 이 모델은 원하는 결과를 줄 수 없다. 지금 문제에서는 그렇지 않으니 계속 진행하자

**2. Exploratory Data Analysis**

> 이제 본격적으로 데이터 분석을 해보자. 데이터는 매우 많다. 이많은 데이터 안에 숨겨진 사실을 찾기 위해선 적절한 시각화가 필요하다.

> 시각화 라이브러리는 mayplotlib, seaborn, plotly 등이 있다. 특정 목적에 맞는 소스코드를 정리해두어 필요할때마다 참고하면 편하다.

**2.1 Pclass**

> 먼저 Pclass 에 대해 살펴보자 Pclass는 ordinal, 서수형 데이터다. 카테고리이며 순서가 있는 데이터 타입이다.

> 먼저 Pclass에 따른 생존률의 차이를 살펴보자. 엑셀의 피벗 차트와 유사한 작업을 하게 되는데 pandas dataframe에서는 groupby를 사용하면 쉽게 가능하다. 또한 pivot이라는 메소드도 있다.

> 'Pclass', 'Survived' 를 가져온 후 pclass로 묶는다. 그러고 나면 각 pclass마다 0,1 이 count가 되고 이를 평균내면 각 pclass별 생존률이 나온다.

> 아래와 같이 count()를 하면, 각 class에 몇명이 있는지 확인 할 수 있으며, sum()을 하면, 216명중 (survived=1) 사람의 총합을 구할수 있다.

In [ ]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).count()


In [ ]:
df_train[['Pclass','Survived']].groupby(['Pclass'],as_index=True).sum()

> pandas의 crosstab을 사용하면 좀더 위 과정을 더 수월하게 볼 수 있다.

In [ ]:
pd.crosstab(df_train['Pclass'], df_train['Survived'], margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
df_train[['Pclass','Survived']].groupby(['Pclass'],as_index=True).mean().sort_values(by='Survived',ascending=False).plot.bar()

> 좀 더 보기 쉽게 그래프를 그려보자. seaborn의 countplot을 이용하면 특정 label에 따른 개수를 확인해볼 수 있다.

In [ ]:
y_position = 1.02
f,ax = plt.subplots(1,2,figsize=(18,8))
df_train['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'],ax=ax[0])
ax[0].set_title('Number of Passengers By Pclass', y=y_position)
ax[0].set_ylabel('Count')
sns.countplot('Pclass',hue='Survived',data=df_train,ax=ax[1])
ax[1].set_title('Pclass: Survived vs Dead', y=y_position)
plt.show()

> 클래스가 높을수록 생존률이 높음을 확인할 수 있다. 이는 생존에 Pclass가 영향을 미치며 나중에 모델을 세울 때 이 feature을 사용하는것이 좋을것이라 판단 할 수 있다.

**2.2 Sex**

> 이번에는 성별로 생존률이 어떻게 달라지는 지 확인해보자

> 마찬가지로 pandas groupby와 seaborn countplot을 사용해 시각화 해보자.

In [ ]:
f, ax = plt.subplots(1,2,figsize=(18,8))
df_train[['Sex','Survived']].groupby(['Sex'],as_index=True).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex',hue='Survived',data=df_train,ax=ax[1])
ax[1].set_title('Sex: Survived vs Dead')
plt.show()

In [ ]:
df_train[['Sex','Survived']].groupby(['Sex'],as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
pd.crosstab(df_train['Sex'],df_train['Survived'],margins=True).style.background_gradient(cmap='summer_r')

> Pclass와 마찬가지로 Sex또한 예측 모델에 쓰일 중요한 feature임을 알 수 있다.

**2.3 Both Sex and Pclass**

> 이버네는 Sex, Pclass 두가지에 관하여 생존이 어떻게 달라지는 지 확인해 보자

> seaborn 의 factorplot을 이용하면 손쉽게 3개의 차원으로 이루어진 그래프를 그릴 수 있다.

In [ ]:
sns.factorplot('Pclass','Survived',hue='Sex',data=df_train,size=6,aspect=1.5)

In [ ]:
sns.factorplot(x='Sex',y='Survived',col='Pclass',data=df_train,satureation=5,size=9,aspect=1)

**2.4 Age**

> 이번에는 Age feature을 살펴보자

In [ ]:
print('제일 나이 많은 탑승객 : {:.1f} Years'.format(df_train['Age'].max()))
print('제일 나이 어린 탑승객 : {:.1f} Years'.format(df_train['Age'].min()))
print('탑승객의 평균 나이 : {:.1f} Years'.format(df_train['Age'].mean()))

> 생존에 따른 Age의 히스토그램을 그려보자

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(df_train[df_train['Survived']==1]['Age'], ax=ax)
sns.kdeplot(df_train[df_train['Survived']==0]['Age'], ax=ax)
plt.legend(['Survived == 1', 'SUrvived == 0'])
plt.show()

> 해석하면 생존자중 나이가 어린사람이 많다.

In [ ]:
# Age distribution within classes
plt.figure(figsize=(8,6))
df_train['Age'][df_train['Pclass']==1].plot(kind='kde')
df_train['Age'][df_train['Pclass']==2].plot(kind='kde')
df_train['Age'][df_train['Pclass']==3].plot(kind='kde')

plt.xlabel('Age')
plt.title('Age Distribution within classes')
plt.legend(['1st Class','2nd Class','3rd Class'])



> Class 와 Age간의 상관관계 확인 가능

> 나이범위를 점점 넓혀가며 생존률이 어떻게 되는지 확인해보자

In [ ]:
cummulate_survival_ratio = []
for i in range(0 , 80):
    cummulate_survival_ratio.append(df_train[df_train['Age']<i]['Survived'].sum()/len(df_train[df_train['Age']<i]['Survived']))

plt.figure(figsize=(7,7))
plt.plot(cummulate_survival_ratio)
plt.title('Survival rate change depending on range of Age', y=1.02)
plt.ylabel('Survival rate')
plt.xlabel('Range of Age(0~x)')
plt.show()

                                    
        

**2.5 Pclass, Sex, Age**

> 지금까지 본 Sex, Pclass, Age, Survived 모두에 대해서 보고싶으면 이를 쉽게 그려주는것이 seaborn의 violinplot 이다.

> x축은 우리가 나눠서 보고싶어하는 case, y축은 보고싶어하는 distribution이다.

In [ ]:
f, ax=plt.subplots(1,2,figsize=(18,8))
sns.violinplot("Pclass","Age",hue="Survived", data=df_train,scale='count',split=True,ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))
sns.violinplot("Sex","Age",hue="Survived",data=df_train,scale="count",split=True,ax=ax[1])
ax[1].set_title("Sex and Age vs Survived")
ax[1].set_yticks(range(0,110,10))
plt.show()

**2.6 Embarked**

> Embarked 는 탑승한 항구를 나타낸다.

In [ ]:
f, ax = plt.subplots(1,1,figsize=(7,7))
df_train[['Embarked','Survived']].groupby(['Embarked'],as_index=True).mean().sort_values(by='Survived',ascending=False).plot.bar(ax=ax)

In [ ]:
f,ax =plt.subplots(2,2,figsize=(20,15))
sns.countplot('Embarked',data=df_train,ax=ax[0,0])
ax[0,0].set_title('(1) No. Of Passengers Boarded')
sns.countplot('Embarked',hue='Sex',data=df_train,ax=ax[0,1])
ax[0,1].set_title('(2) Male_Femaile Split for Embarked')
sns.countplot('Embarked',hue="Survived",data=df_train,ax=ax[1,0])
ax[1,0].set_title('(3) Embarked vs Survived')
sns.countplot('Embarked',hue='Pclass',data=df_train,ax=ax[1,1])
ax[1,1].set_title('(4) Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

> (1) 전체적으로 보면 S에 가장 많은 사람이 탑승

> (2) C와 Q는 남여 비율이 비슷하지만, S는 남자가 더 많다.

> (3) 생존확률이 S일 경우 낮다.

> (4) Class로 Split해서 보니, C가 생존확률이 높은것은 클래스가 높은 사람이 많이 타서 그렇다. S는 3rd Class가 많아 생존확률이 낮게 나왔다.

**2.6 Family - SibSp ( 형제, 자매 ) + Parch ( 부모, 자녀 )**

> SibSp와 Parch를 합하면 Family가 된다. Family로 합쳐서 분석해봅시다.

In [ ]:
df_train['FamilySize']=df_train['SibSp']+df_train['Parch']+1
df_test["FamilySize"]=df_test['SibSp']+df_train['Parch']+1

print('Maximum size of Family : ', df_train['FamilySize'].max())
print('Minimum size of Family : ', df_train['FamilySize'].min())

In [ ]:
f,ax = plt.subplots(1,3,figsize=(40,10))
sns.countplot('FamilySize', data=df_train,ax=ax[0])
ax[0].set_title('(1) No. Of Passengers Boarded', y=1.02)

sns.countplot('FamilySize', hue='Survived',data=df_train,ax=ax[1])
ax[1].set_title('(2) Survived countplot depending on FamilySize',  y=1.02)


df_train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=True).mean().sort_values(by='Survived', ascending=False).plot.bar(ax=ax[2])
ax[2].set_title('(3) Survived rate depending on FamilySize',  y=1.02)

plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

**2.8 Fare**

> 탑승요금을 histogram으로 표현해보자

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
g=sns.distplot(df_train['Fare'],color='b',label='Skwness : {:.2f}'.format(df_train['Fare'].skew()),ax=ax )
g=g.legend(loc='best')

> distribution이 매우 비대치인것을 알 수 있다. 만약 이대로 모델에 넣어준다면 자칫 모델이 잘못 학습할 수도 있다.

> 몇개 없는 outlier에 대해 너무 민감하게 반응한다면 실제 예측 시 좋지 못한 결과를 부를 수 있다.

> outlier의 영향을 줄이기 위해 log 분포를 사용하자

> 여기서 우리는 pandas의 유용한 기능을 사용한다. dataFrame의 특정 columns에 공통된 작업을 적용하고 싶다면 아래의 map 또는 apply를 사용하면 된다.

> 우리가 여기서 원하는 것은 Fare colums의 데이터에 log취하는 것인데 lambda 함수를 이용해 간단한 로그를 적용하는 함수를 map에 인수로 넣어준다면 적용이 된다.

In [ ]:
df_test.loc[df_test.Fare.isnull(), 'Fare'] = df_test['Fare'].mean()

df_train['Fare']=df_train['Fare'].map(lambda i: np.log(i) if i>0 else 0)
df_test['Fare']=df_test['Fare'].map(lambda i: np.log(i) if i>0 else 0)

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(8,8))
g=sns.distplot(df_train['Fare'], color='b',label='Skewness : {:.2f}'.format(df_train['Fare'].skew()),ax=ax)
g=g.legend(loc='best')

> 이와 같이 데이터를 학습하기에 적합한 값으로 조정하는 것을 feature engineering 이라 한다.

**2.9 Cabin**

> 이 feature은 NaN이 대략 80%이므로 중요한 정보를 얻어내기 쉽지않다.

> 그러므로 모델에 포함시키지 않는것이 현명

In [ ]:
df_train.head()

**2.10 Ticket**

>> 다양한 값이 포함되어 있으니 일단 확인을 해보자

In [ ]:
df_train['Ticket'].value_counts()

> 서로 다른것들이 너무 많으니 이에대해 예측할 모형을 만들기엔 적합하지 않으므로 넘어가자